# Quantum Annealer for Philogenetic Trees

---

In this notebook, we are going to use D-Wave's Ocean to create the needed optimization problem to reconstruct Philogenetic Trees. In short, this is where the real work begins. This work will be based in 2 documents, a book that describes Quantum Annealing [1], and the paper for reconstruction of Philogenetic Trees [2].



[1] Combarro, E. F., & Gonzalez-Castillo, S. (2023). A practical guide to quantum machine learning and quantum optimisation: Hands-On Approach to Modern Quantum Algorithms. Packt Publishing.

[2] Onodera, W., Hara, N., Aoki, S., Asahi, T., & Sawamura, N. (2022). Phylogenetic tree reconstruction via graph cut presented using a quantum-inspired computer. Molecular Phylogenetics and Evolution, 178, 107636. https://doi.org/10.1016/j.ympev.2022.107636

In [4]:
import dimod
from dwave.system import DWaveSampler, EmbeddingComposite

First, we start with an example

In [5]:
# Coefficients of the quadratic term elements (squared or products)
J = {(0,1):1, (0,2):1}
# Coefficients of the linear terms
h = {}
problem = dimod.BinaryQuadraticModel(h, J, 0.0, dimod.SPIN)
print("The problem we are going to solve is:")
print(problem)

The problem we are going to solve is:
BinaryQuadraticModel({0: 0.0, 1: 0.0, 2: 0.0}, {(1, 0): 1.0, (2, 0): 1.0}, 0.0, 'SPIN')


From the paper we have that the minimization is defined as:

$$Min_{cut}=\sum_{i=1}^{n-1}\sum_{j=i+1}^n d_{ij}(x_i-x_j)^2,\qquad x_i=\{0,1\}, \quad i = 1,...,n.$$

Where $d_{ij}$ is the element $ij$ from the matrix $D$, where the differences between elements are represented. In other words, if you consider the problem as a graph, $D$ is the adjacency matrix from the graph.

If we take a closer look at this formula, we can see that we take the top part of the matrix, also, we start counting from 1, and I don't want that, so we can rewrite the expression as:

$$Min_{cut}=\sum_{i=0}^{n-2}\sum_{j=i+1}^{n-1} d_{ij}(x_i-x_j)^2,\qquad x_i=\{0,1\}, \quad i = 0,...,n-1.$$